In [29]:
!pip install unidecode

   -------------------------------------- 235.5/235.5 kB 719.7 kB/s eta 0:00:00


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [77]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import unidecode
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
import time
import numpy as np

In [2]:
item_titles = pd.read_csv("items_titles.csv")

In [3]:
item_titles

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...
...,...
29995,Tênis Vans Old Skool I Love My Vans - Usado - ...
29996,Tênis Feminino Preto Moleca 5296155
29997,Tenis Botinha Com Pelo Via Marte Original Lanç...
29998,Tênis Slip On Feminino Masculino Original Sapa...


### Primeiro vamos processar o texto
- Lower case
- Remoção de pontuação
- Remoção de numeros
- Remoção de acentuação

In [4]:
def process_text(text):
    lower_text = text.lower()
    no_pontuation_text = re.sub(r"[^\w\s]", "", lower_text)
    no_number_text = re.sub("\d+", "", no_pontuation_text)
    no_accents_text = unidecode.unidecode(no_number_text)
    
    return no_accents_text

In [5]:
processed_item_titles = item_titles.apply(lambda x: process_text(x.ITE_ITEM_TITLE), axis=1)

### Vetorização
- Vamos aplicar um TF-IDF para vetorizar os texto
- Utiliza uma contagem de tokens de cada documento (produto) e normaliza o termo de acordo com a ocorrência em todo o corpus. Sendo assim termos mais relevantes possuem um maior peso na vetorização.
- Aplicamos <b>fit_transform</b> na base de treino e somente <b>transform</b> na base de teste

In [6]:
tfidf = TfidfVectorizer()

In [7]:
tfidf_matrix = tfidf.fit_transform(processed_item_titles)

In [9]:
feature_names = tfidf.get_feature_names_out()

In [10]:
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

### Primeira abordagem - nested for loops
- Abordagem mais lenta
- Iteramos um dataframe pandas usando Iterrows, porém precisamos iterar o dataframe contra ele mesmo
- Complexidade: Aproximadamente O(cosine_similarity*numero_produtos^2)
- Tempo de execução por produto (um contra todos) na base de treino: entre 15 a 20 segundos

In [11]:
def nested_loops_computation(df_tfidf, num_itens = 1):
    start_time = time.time()
    output_list = []
    output = pd.DataFrame(columns=["item_title1", "item_title2", "Score"])
    for index, item in enumerate(df_tfidf.head(num_itens).iterrows()):
        for index2, item2 in enumerate(df_tfidf.iterrows()):
            item_title1 = processed_item_titles[index]
            item_title2 = processed_item_titles[index2]
            score = cosine_similarity(item[1].values.reshape(1,-1),item2[1].values.reshape(1,-1))[0]
            output_list.append([item_title1, item_title2, score])
    output_df = pd.DataFrame(output_list, columns=["item_title1", "item_title2", "Score"])
    print("--- %s seconds ---" % (time.time() - start_time))
    return output_df

In [85]:
nested_loops_computation(df_tfidf).sort_values(by = "Score", ascending = False)

--- 28.832355499267578 seconds ---


,item_title1,item_title2,Score
0,tenis ascension posh masculino preto e vermelho,tenis ascension posh masculino preto e vermelho,[0.9999999999999999]
14384,tenis ascension posh masculino preto e vermelho,tenis masculino ascension caminhada barato pre...,[0.5848281533383759]
12706,tenis ascension posh masculino preto e vermelho,tenis ascension calce facil confortavel mascul...,[0.40127400839073846]
11756,tenis ascension posh masculino preto e vermelho,tenis masculino ascension bx preto e marinho,[0.3985348799922064]
15825,tenis ascension posh masculino preto e vermelho,tenis feminino ascension academia caminhada c...,[0.3597332901387262]
...,...,...,...
5937,tenis ascension posh masculino preto e vermelho,bicicleta aro bicolor menino ultra bikes infa...,[0.0]
15801,tenis ascension posh masculino preto e vermelho,botinha academia feminino treino fitness couro...,[0.0]
15803,tenis ascension posh masculino preto e vermelho,bicicleta urban gazelle holandesa,[0.0]
15805,tenis ascension posh masculino preto e vermelho,sapatilha ciclismo shimano speed rp rp preta,[0.0]


### Segunda abordagem - cdist 
- Abordagem abstrai um código em C e utiliza numpy através do pacote SciPy, função cdist
- Pesquisando, ainda parece ser um código  de complexidade O(numero_produtos^2). A otimização vem dessa abstração dita acima https://stackoverflow.com/questions/51630056/why-cdist-from-scipy-spatial-distance-is-so-fast
- Cada execução item pouco menos que 1 segundo (0.96s)

In [82]:
def cidst_computation(df_tfidf, processed_item_titles, num_itens = 1):
    start_time = time.time()
    numpy_tfidf = df_tfidf.to_numpy()
    distances = cdist(numpy_tfidf[0:num_itens], numpy_tfidf, metric='cosine').flatten()
    distances_norm = 1-distances #precisamos inverter para satisfazer a condição de que "quanto maior melhor"
    text_df = pd.DataFrame(processed_item_titles)
    text_df = text_df.head(num_itens).merge(text_df, how="cross")
    text_df["score"] = distances_norm
    print("--- %s seconds ---" % (time.time() - start_time))
    return text_df

In [87]:
cidst_computation(df_tfidf, processed_item_titles, num_itens = 3).sort_values(by = "score", ascending = False)

--- 1.571019172668457 seconds ---


,0_x,0_y,score
0,tenis ascension posh masculino preto e vermelho,tenis ascension posh masculino preto e vermelho,1.000000
60002,tenis feminino le parc hocks blackice original...,tenis feminino le parc hocks blackice original...,1.000000
30001,tenis para caminhada super levinho spider corr...,tenis para caminhada super levinho spider corr...,1.000000
56365,tenis para caminhada super levinho spider corr...,tenis masculino caminhada levinho spider acade...,0.716252
80377,tenis feminino le parc hocks blackice original...,tenis feminino hocks skatista le parc branco e...,0.715609
...,...,...,...
58902,tenis para caminhada super levinho spider corr...,sapatilha sidi shot air carbon edicao limitada,0.000000
58901,tenis para caminhada super levinho spider corr...,sandalia masculina stock sandals oberyn frete ...,0.000000
58898,tenis para caminhada super levinho spider corr...,sapatilha mtb shimano tamanho eur,0.000000
14381,tenis ascension posh masculino preto e vermelho,sapatilha para mountain bike bontrager cadence...,0.000000


### Terceira Abordagem - Implementar a distância de cosenos usando numpy
- Implementação do calculo da distância de cosenos
- Uma propriedade do TF-IDF é que ele já retorna vatores com tamanho unitário, portanto somente o produto escalar entre os vatores é suficiente
- np.dot é eficiente pois faz chamadas de operações computacionais baixo nivel (https://www.quora.com/Which-method-does-NumPy-use-to-calculate-dot-product-at-such-high-speeds)
- Essa sem dúvida foi a abordagem mais rápida, conseguindo processar 1000 itens em 32 segundos

In [110]:
def cosine_distance_formula(df_tfidf, processed_item_titles, num_itens=1):
    start_time = time.time()
    numpy_tfidf = df_tfidf.to_numpy()
    distances = np.dot(numpy_tfidf[0:num_itens],numpy_tfidf.T).flatten()
    text_df = pd.DataFrame(processed_item_titles)
    text_df = text_df.head(num_itens).merge(text_df, how="cross")
    text_df["score"] = distances
    print("--- %s seconds ---" % (time.time() - start_time))
    return text_df

In [113]:
cosine_distance_formula(df_tfidf, processed_item_titles, num_itens = 1000).sort_values(by = "score", ascending = False)

--- 32.49164891242981 seconds ---


,0_x,0_y,score
20190673,tenis olympikus asas feminino de corrida original,tenis olympikus asas feminino de corrida original,1.0
27210907,tenis feminino casual moleca super confortavel,tenis feminino casual moleca super confortavel,1.0
14520484,tenis feminino allstars unissex coloridos envi...,tenis feminino allstars unissex coloridos envi...,1.0
17880596,calcado de crianca menino com luzinha de led e...,calcado de crianca menino com luzinha de led e...,1.0
29160972,tenis hocks de skate flat lite mint masculino ...,tenis hocks de skate flat lite mint masculino ...,1.0
...,...,...,...
14600902,nike air force shadow magic ember tamanho d...,tenis feminino casual de veludo marsala estilo...,0.0
14600903,nike air force shadow magic ember tamanho d...,tenis slink infantil snj,0.0
14600904,nike air force shadow magic ember tamanho d...,tenis de malha knit na cor nude,0.0
14600905,nike air force shadow magic ember tamanho d...,bicicleta passeio feminina aro retro cesta vim...,0.0


### Sanity Check - As abordagens são consistentes entre si?
- Para 1 item elas retornam o mesmo Top 5 com o mesmo Score, como mostrado abaixo

In [114]:
output_nested_one_item = nested_loops_computation(df_tfidf).sort_values(by = "Score", ascending = False)
output_cdist_one_item = cidst_computation(df_tfidf, processed_item_titles).sort_values(by = "score", ascending = False)
output_cosine_one_item = cosine_distance_formula(df_tfidf, processed_item_titles).sort_values(by = "score", ascending = False)

--- 33.25049614906311 seconds ---
--- 0.9986593723297119 seconds ---
--- 0.4693763256072998 seconds ---


In [115]:
output_nested_one_item.head()

,item_title1,item_title2,Score
0,tenis ascension posh masculino preto e vermelho,tenis ascension posh masculino preto e vermelho,[0.9999999999999999]
14384,tenis ascension posh masculino preto e vermelho,tenis masculino ascension caminhada barato pre...,[0.5848281533383759]
12706,tenis ascension posh masculino preto e vermelho,tenis ascension calce facil confortavel mascul...,[0.40127400839073846]
11756,tenis ascension posh masculino preto e vermelho,tenis masculino ascension bx preto e marinho,[0.3985348799922064]
15825,tenis ascension posh masculino preto e vermelho,tenis feminino ascension academia caminhada c...,[0.3597332901387262]


In [116]:
output_cdist_one_item.head()

,0_x,0_y,score
0,tenis ascension posh masculino preto e vermelho,tenis ascension posh masculino preto e vermelho,1.000000
14384,tenis ascension posh masculino preto e vermelho,tenis masculino ascension caminhada barato pre...,0.584828
12706,tenis ascension posh masculino preto e vermelho,tenis ascension calce facil confortavel mascul...,0.401274
11756,tenis ascension posh masculino preto e vermelho,tenis masculino ascension bx preto e marinho,0.398535
15825,tenis ascension posh masculino preto e vermelho,tenis feminino ascension academia caminhada c...,0.359733


In [117]:
output_cosine_one_item.head()

,0_x,0_y,score
0,tenis ascension posh masculino preto e vermelho,tenis ascension posh masculino preto e vermelho,1.000000
14384,tenis ascension posh masculino preto e vermelho,tenis masculino ascension caminhada barato pre...,0.584828
12706,tenis ascension posh masculino preto e vermelho,tenis ascension calce facil confortavel mascul...,0.401274
11756,tenis ascension posh masculino preto e vermelho,tenis masculino ascension bx preto e marinho,0.398535
15825,tenis ascension posh masculino preto e vermelho,tenis feminino ascension academia caminhada c...,0.359733


## Aplicação na Base de Teste

In [121]:
test_set = pd.read_csv("items_titles_test.csv")

In [122]:
processed_item_titles_test = test_set.apply(lambda x: process_text(x.ITE_ITEM_TITLE), axis=1)

In [129]:
tfidf_test = pd.DataFrame(processed_item_titles_test, columns= ["ITE_ITEM_TITLE"]).apply(lambda x: tfidf.transform(x))

In [130]:
df_tfidf_test = pd.DataFrame(tfidf_test.ITE_ITEM_TITLE.toarray(), columns=feature_names)

In [134]:
output_for_test_set = cosine_distance_formula(df_tfidf_test,processed_item_titles_test,num_itens=10000)

--- 262.3928048610687 seconds ---


In [135]:
output_for_test_set

,0_x,0_y,score
0,tenis olympikus esporte valente masculino kids,tenis olympikus esporte valente masculino kids,1.000000
1,tenis olympikus esporte valente masculino kids,bicicleta barra forte samy c marchas cubo c r...,0.000000
2,tenis olympikus esporte valente masculino kids,tenis usthemp slipon tematico labrador,0.009220
3,tenis olympikus esporte valente masculino kids,tenis casual feminino moleca tecido tie dye,0.009180
4,tenis olympikus esporte valente masculino kids,tenis star baby sapatinho conforto brinde,0.008600
...,...,...,...
99999995,tenis polo ralph lauren modelo cantor low bran...,chuteira futsal oxn velox infantil,0.000000
99999996,tenis polo ralph lauren modelo cantor low bran...,sapatenis casual masculino estiloso horas conf...,0.031383
99999997,tenis polo ralph lauren modelo cantor low bran...,tenis feminino infantil molekinha tie dye,0.008115
99999998,tenis polo ralph lauren modelo cantor low bran...,tenis feminino leve barato ganhe colchonete p...,0.005413


### Considerações Finais
1. Implementação usando loops aninhados não é eficiente
2. Implementação utilizando cdist é mais rápida que a implementação de loops, porém não pareceu escalável.
3. A implementação direta utilizando np.dot foi a unica que consegui processar o tamanho todo da lista na minha máquina local
4. A solução utilizando np.dot é escalável e foi utilizada para o calculo final
5. Foi entregue, conforme solicitado:
    - Um modelo treinado com a base de treino (tf-idf)
    - Base de teste com os scores de todos os itens contra todos os itens (output_for_test_set)
    - Três abordagens possíveis e discussão sobre tempo de execução